<a href="https://colab.research.google.com/github/Zenith40/Demo-Code/blob/main/Fake_News_Prediction_Using_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
# Importing Data From Kaggle

In [103]:
# Create API token on Kaggle and download the .json file and upload .json file to colab, then set directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [104]:
# Download the Dataset (https://www.kaggle.com/c/fake-news/data?select=train.csv) == c/fake-news
!kaggle competitions download -c fake-news

fake-news.zip: Skipping, found more recently modified local copy (use --force to force download)


In [105]:
# Unzip the file to use the dataset
import zipfile
zip_ref = zipfile.ZipFile('/content/fake-news.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [106]:
# Importing Libraries

import numpy as np
import pandas as pd
import re   # Useful for searching text in a document
import nltk # Natural Language Toolkit
from nltk.corpus import stopwords    #removes common words from the document : the,is,was etc.
from nltk.stem.porter import PorterStemmer   # removes prefix and suffix to make the words same : call,called,calling -> call
from sklearn.feature_extraction.text import TfidfVectorizer # Converts words into numbers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [107]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [108]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [109]:
# Data Preprocessing

In [110]:
df = pd.read_csv('/content/train.csv')

In [111]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [112]:
df.shape

(20800, 5)

In [113]:
df.isnull().sum()*100/df.shape[0]

,0
id,0.000000
title,2.682692
author,9.408654
text,0.187500
label,0.000000


In [114]:
# Replacing null values  with emppty string

df = df.fillna('')

In [115]:
# Merging the title, author column

df['content'] = df['author'] + ' '+ df['title']  # +df['text']

In [116]:
port_stem = PorterStemmer()

In [117]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #filters the numbers and special characters keeping only the alphabets
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [118]:
df['content'] = df.content.apply(stemming)

In [119]:
print(df.content)

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [120]:
# Seperating content and label

X = df.content.values
Y = df['label']

In [121]:
# Converting textual data into numerical data

vectorizer = TfidfVectorizer() # TERM FREQUENCY, INVERSE DOCUMENT FREQUENCY = Tfidf

x = vectorizer.fit_transform(X)

# Refer Documentation or geeks-for-geeks : https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html OR https://www.geeksforgeeks.org/understanding-tf-idf-term-frequency-inverse-document-frequency/

# print(vectorizer.vocabulary_)  -> Word Indexing

In [122]:
# Splitting Data into training and testing

x_train,x_test,y_train,y_test = train_test_split(x,Y,test_size=0.2,stratify=Y,random_state=2)

In [123]:
# Training Logistic Regression Model

model = LogisticRegression()

In [124]:
model.fit(x_train,y_train)

LogisticRegression()

In [125]:
# Evalualtion

accuracy_score(model.predict(x_train),y_train) # accuracy score on training data

0.9863581730769231

In [126]:
# accuracy score on testing data

accuracy_score(model.predict(x_test),y_test)

0.9790865384615385

In [127]:
# Making a Predictive System

x_new = x_test[0]

prediction = model.predict(x_new)
print(prediction)

if prediction[0] == 0:
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [128]:
y_test[0]

1

In [129]:
from sklearn.model_selection import cross_val_score

cross_val_score(model,x_train,y_train, cv=5,scoring='accuracy').mean()

0.973497596153846

In [130]:
# Training on Whole Dataset

cross_val_score(model,x,Y, cv=5,scoring='accuracy').mean()

0.9763461538461538

In [131]:
model.fit(x,Y)

LogisticRegression()

In [ ]:
#test = pd.read_csv('/content/test.csv')

In [133]:
#test = test.fillna('')
#test['content'] = test['author'] + ' '+ test['title']  # +df['text']
#test['content'] = test.content.apply(stemming)
#X_test = test.content.values
#X_test = vectorizer.transform(X_test)

In [142]:
#model.predict(X_test)

array([0, 1, 1, ..., 0, 1, 0])